In [1]:
import os 
import shutil
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
import sys
sys.path.append('..')
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from training.Helpers import metrics,make_model_cv,load_cv_data,take_params,test_cv_model
from tqdm import tqdm

In [2]:
#dataprep

# desktop = pathlib.Path('../../pics_dataset/')
# ids = [ [x,int(x.name.split('_')[1])] for x in  list(desktop.rglob("*.png"))]
# data = pd.DataFrame(ids,columns=['path','id'])
# data
# y = pd.read_pickle('../data/datasets/train/coding_pic.pkl')
# x = pd.read_pickle('../data/datasets/train/data_40k.pkl')
# x = x.loc[x.index.isin(data.id)]
# y = y.loc[y.index.isin(data.id)]
# _,_,y_train,y_test = train_test_split(x,y,test_size=0.1)
# len(y_train),len(y_test)
# for sample in [1000,2000,5000,8000,10000,15000,20000]:
#     _ = y_train.to_frame().groupby('hotel_cluster').apply(lambda x: x.sample(frac=(sample/len(y_train)))).droplevel(level=0)
#     os.mkdir(f'../../pics_sample_{sample}')
#     for i in y_train.unique():
#         os.mkdir(f'../../pics_sample_{sample}/{i}')
        
#     for i,d in data.loc[data.id.isin(_.index)].iterrows():
#         cat = _.loc[_.index == d.id].values[0][0]
#         #print(d['path'].name)
#         name = d['path'].name
#         shutil.copy(d['path'],f'../../pics_sample_{sample}/{cat}/{name}')
        
# for i in y.unique():
#     os.mkdir(f'../../pics_test/{i}')        
    
# for i,d in data.loc[data.id.isin(y_test.index)].iterrows():
#     cat = y_test.loc[y_test.index == d.id].values[0]
#     #print(d['path'].name)
#     name = d['path'].name
#     shutil.copy(d['path'],f'../../pics_test/{cat}/{name}')

In [2]:
pos =  {
    'dense1':[32,64,173,128,256],
    'dense2':[64,128,256,512],
    'drop2':[0.1,0],
    'dense3':[64,128,256,512,1024],
    'drop3':[0.1,0,0.2],
    'dense4':[64,128,256,512,1024],
    'drop4':[0.1,0,0.2],
    'dense5':[64,128,256],
    'dense6':[64,128,256],
    'drop6':[0.1,0,0.2],
    'bn':[0,1]
    
}


In [3]:
diff_params = take_params(pos,1000)
pd.DataFrame(diff_params).to_pickle(f'../results/params_cv.pkl')

In [4]:
results = []
best_acc_30k = -1
for idx,params in enumerate(diff_params):
    print(f"itteration:{idx} \n")
    history = []
    for sample_size in [1000,2000,5000,8000,10000,15000,20000,30000]:
        train_ds,val_ds = load_cv_data(sample_size)
        model = make_model_cv(params)
        h = model.fit(train_ds, batch_size=128, epochs=100, validation_data=val_ds, verbose = 0,
            callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True,start_from_epoch=30)])
        
        h2  = {
            'val_loss':h.history['val_loss'],
            'val_accuracy':h.history['val_accuracy'],
            'loss':h.history['loss'],
            'accuracy':h.history['accuracy'],
            'sample_size' : sample_size,
        }
        
        history.append(h2)
        print('tutaj')
        res = test_cv_model(model)
        res['sample_size'] = sample_size
        res['params_id'] = idx
        results.append(res)
    
    pd.DataFrame(results).to_pickle(f'../results/res_cv_.pkl')
    if res['accuracy_score'] > best_acc_30k:
        best_acc_30k = res['accuracy_score']
        pd.DataFrame(history).to_pickle(f'../results/best_hist_cv.pkl')

itteration:0 

Found 8001 files belonging to 99 classes.
Using 5601 files for training.
Using 2400 files for validation.
44/44 [==============================] - 8s 97ms/step - loss: 4.9133 - accuracy: 0.0157 - val_loss: 4.6292 - val_accuracy: 0.0075
tutaj
tutaj
Found 3250 files belonging to 99 classes.
Found 10001 files belonging to 99 classes.
Using 7001 files for training.
Using 3000 files for validation.
55/55 [==============================] - 15s 208ms/step - loss: 4.8824 - accuracy: 0.0147 - val_loss: 4.6202 - val_accuracy: 0.0153
tutaj
tutaj
Found 3250 files belonging to 99 classes.
Found 15000 files belonging to 99 classes.
Using 10500 files for training.
Using 4500 files for validation.
83/83 [==============================] - 21s 206ms/step - loss: 4.8005 - accuracy: 0.0249 - val_loss: 4.6133 - val_accuracy: 0.0198
tutaj
tutaj
Found 3250 files belonging to 99 classes.


KeyboardInterrupt: 